In [11]:
import paramiko
import os

# Define paths and server details
local_path = "C:/Users/admin/Desktop/odoo-16/extra-addons"
remote_temp_path = "/tmp/custom_accounting_fix"
server_ip = "146.88.26.77"
server_port = 2233
username = "root"
container_name = "odoo16-odoo-16"
container_path = "/usr/lib/python3/dist-packages/odoo/addons/"

# Initialize SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    # Connect to the server
    ssh.connect(server_ip, port=server_port, username=username, password="di#DWa2W%tR6Sd@FI", timeout=10)


    # Use SCP to copy local folder to remote temp folder
    sftp = ssh.open_sftp()

    # Recursively copy local directory to remote temporary directory
    def recursive_upload(local, remote):
        if os.path.isdir(local):
            try:
                sftp.mkdir(remote)
            except IOError:
                pass  # Ignore if the folder already exists
            for item in os.listdir(local):
                recursive_upload(os.path.join(local, item), f"{remote}/{item}")
        else:
            sftp.put(local, remote)
            print(f"Copied {local} to {remote}")

    # Copy to the remote temporary directory
    print("Starting upload to the remote server's temporary directory...")
    recursive_upload(local_path, remote_temp_path)
    print(f"All files uploaded to the remote server's temporary directory: {remote_temp_path}")

    # Copy the whole folder from remote temp directory to Docker container
    print(f"Copying folder {remote_temp_path} to container {container_name}:{container_path}...")
    copy_command = f"docker cp {remote_temp_path} {container_name}:{container_path}"
    ssh.exec_command(copy_command)
    print(f"Folder copied to container location: {container_name}:{container_path}custom_accounting_fix")

    # Remove the temporary folder on the remote server
    print(f"Removing the temporary folder from the remote server: {remote_temp_path}...")
    ssh.exec_command(f"rm -rf {remote_temp_path}")
    print("Temporary folder removed from the remote server.")

    # Restart the Docker container
    print(f"Restarting Docker container: {container_name}...")
    restart_command = f"docker restart {container_name}"
    stdin, stdout, stderr = ssh.exec_command(restart_command)
    stdout_lines = stdout.readlines()
    stderr_lines = stderr.readlines()
    if stdout_lines:
        print(f"Docker container restarted successfully: {''.join(stdout_lines)}")
    if stderr_lines:
        print(f"Errors during container restart: {''.join(stderr_lines)}")

finally:
    # Close SFTP and SSH connections
    sftp.close()
    ssh.close()

Starting upload to the remote server's temporary directory...
Copied C:/Users/admin/Desktop/odoo-16/extra-addons\.DS_Store to /tmp/custom_accounting_fix/.DS_Store
Copied C:/Users/admin/Desktop/odoo-16/extra-addons\accounting_pdf_reports\accounting_pdf_reports\data\account_account_type.xml to /tmp/custom_accounting_fix/accounting_pdf_reports/accounting_pdf_reports/data/account_account_type.xml
Copied C:/Users/admin/Desktop/odoo-16/extra-addons\accounting_pdf_reports\accounting_pdf_reports\doc\RELEASE_NOTES.md to /tmp/custom_accounting_fix/accounting_pdf_reports/accounting_pdf_reports/doc/RELEASE_NOTES.md
Copied C:/Users/admin/Desktop/odoo-16/extra-addons\accounting_pdf_reports\accounting_pdf_reports\i18n\ar.po to /tmp/custom_accounting_fix/accounting_pdf_reports/accounting_pdf_reports/i18n/ar.po
Copied C:/Users/admin/Desktop/odoo-16/extra-addons\accounting_pdf_reports\accounting_pdf_reports\i18n\ar_001.po to /tmp/custom_accounting_fix/accounting_pdf_reports/accounting_pdf_reports/i18n/a

FileNotFoundError: [Errno 2] No such file